In [25]:
# based on https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html

from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils
from keras.datasets import cifar10

from sklearn.metrics import log_loss

import numpy as np

In [27]:
import cv2

from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

nb_train_samples = 3000 # 3000 training samples
nb_valid_samples = 100 # 100 validation samples
num_classes = 10

row_size, col_size = 224, 224

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_dim_ordering() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid

X_train,y_train,X_val,y_val = load_cifar10_data(row_size,col_size)

In [33]:
from keras.models import model_from_json, load_model
import random as rand
# temporary value to hold while we debug
i = 0

# 1) decodes the algorithm solution to get hyperparameters
# 2) trains the individual model
# 3) calculates fitness function on validation set and returns fitness score
def train_evaluate(ga_individual_solution):
    # convert model to json
    individual = {"name":i,"layers":ga_individual_solution}
    print(ga_individual_solution.__dict__)
    # Train model and predict on validation set
    model = model_from_json(ga_individual_solution)
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
    y_pred = model.predict(X_val)
    
    # Calculate the accuracy score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    return rmse,

# 1) mutates a models's parameters based on probability in range [0,1.0]
# 2) if given a targeted list of attributes, only mutate those; otherwise attempt mutation on all
def mutate_model(model, mutate_prob, targeted_attr=None):
    """
    Args:     model (dict), mutate_prob(float), targeted_attributes(dict{key=str,value=float})
    Output:   model (dict)
    """
    
    # FOR NOW, WE'RE ONLY MUTATING THE FILTER
    print("mutating model")
    for layer,attrs in model.items():
        random.seed(9001)
        print("layer: ",layer)
        for param in attrs:
            mutate = rand.uniform(0.0,1.0)
            if mutate > mutate_prob:
                #if param=="type" and attrs[param] != "Input":
                #    print("mutating type ",attrs[param])
                #    model[layer[param]] = rand.choice("Convolution","Pooling","Softmax")
                if param=="filter":
                    vals = attrs[param]
                    result = []
                    for i in range(0,len(attrs[param])):
                        # DISTRIBUTION APPROACH
                        #mean,std_dev,n = vals[i],0.3,30
                        #distribution = np.random.normal(mean,std_dev*mean,n)
                        #newval = int(round(abs(random.choice(distribution))))
                        
                        # FACTOR OF TWO APPROACH
                        result.insert(i,int(round(vals[i]*rand.choice([0.5,1.0,2.0]))))
                    result[1] = result[0]
                    attrs[param] = result
    return model

# load in initial data from json
JSON_PATH = "/home/collin/Desktop/EA/evnet/model-json/"

with open(JSON_PATH+"vgg16.json", "r") as network_data:
    network = json.load(network_data)
    
layer_data = network["layers"]

# test mutation
#mutate_model({layer:values for (layer,values) in layer_data.items()},0.5,)


In [35]:
import random
import json
import array

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import pprint


# We want to maximize accuracy
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Initialization functions
def initPopulation(pcls, ind_init, initial_data, pop_size):
    result = []
    for i in range(0,pop_size):
        result.insert(i,ind_init(initial_data))
    return result

population_size = 5
num_generations = 10

# Instantiate the initializers from json layer information
toolbox.register("population", initPopulation, list, creator.Individual, layer_data, population_size)

toolbox.register('mate', tools.cxOnePoint)
toolbox.register('mutate', mutate_model, creator.Individual, 0.5)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population()
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)


{'fitness': deap.creator.FitnessMax(())}


TypeError: the JSON object must be str, not 'Individual'

In [ ]:
# Print top N solutions
best_individuals = tools.selBest(population,k = 2)
print('\nMost fit are: ')
for bi in best_individuals:
    print('\n me')

In [2]:
# Train the model using best configuration on complete training set 
#and make predictions on the test set
X_train, y_train, X_test, y_test = load_cifar10_data(test_data,best_window_size)

inputs = Input(shape=(best_window_size,1))
x = LSTM(best_num_units, input_shape=(best_window_size,1))(inputs)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs = inputs, outputs = predictions)
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)

NameError: name 'test_data' is not defined